In [1]:
# Imports
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

import torch
import pathlib
import numpy as np
import pandas as pd
import utils
import config
# sys.path.append('./baseline_src')

import ray 
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch
from ray import tune
import json
from statistics import mean
# from config import DynamicGraphTemporalSignal_custom
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch
import joblib
import random
from torch.utils.data import DataLoader, TensorDataset
import joblib
import random

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

device = 'cpu'

num_columns = ['precipitation_intensity','temperature','humidity','average_speed','extreme_congestion','past_week_load_21', # NODE
               'distance_traveled',
            #    'total_traveled','median_income_last12months','white_pct','black_pct', # EDGE
            #    'hispanic_pct','public_transit_pct','pct_public_transit_for_work' # EDGE
               ]
ohe_columns = ['route_id','is_holiday','is_weekend','route_direction_name']
cat_columns = ['time_window_cat','stop_sequence_cat','year_cat','month_cat']

TARGET = 'trip_load_4_bin_transit'

REQ_COLS = num_columns + ohe_columns + cat_columns + [TARGET]

/Users/samirgupta/Desktop/WORK/--_work/Research/After_12th_December_2023/05_GNN_occu_prediction/WeGo_Occu_pred/NEW_FOLDER/CODE/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-28 19:26:39,927	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-28 19:26:40,047	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def train_one_epoch(model,train_loader,focal_loss,optimizer):
    model.train()

    time = 0
    loss = 0
    for inputs, targets in train_loader:
        inputs.to(device)
        targets.to(device)
        outputs = model(inputs)
        # targets = np.argmax(targets,axis=-1)
        loss = loss + focal_loss(outputs, targets)
        time += 1
    loss = loss / (time+1) #Getting the mean loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    return loss

def validate_one_epoch(model,validation_loader,focal_loss,optimizer):
    model.eval()
    time = 0
    loss = 0
    for inputs, targets in validation_loader:
        inputs.to(device)
        targets.to(device)
        outputs = model(inputs)
        loss = loss + focal_loss(outputs, targets)
        time += 1
    loss = loss / (time+1) #Getting the mean loss

    return loss

In [3]:

def train_and_evaluate_model(model, epochs, patience, train_loader, validation_loader, focal_loss, optimizer):

    early_stopper = utils.EarlyStopper(patience=patience, min_delta=0)

    history = {
        'epoch':[],
        'train_loss':[],
        'validation_loss':[],
    }    # Training loop

    for epoch in tqdm(range(epochs)):
        train_loss = train_one_epoch(model,train_loader,focal_loss,optimizer)
        validation_loss = validate_one_epoch(model,validation_loader,focal_loss,optimizer)

        val_loss_to_monitor = validation_loss

        history['epoch'].append(epoch+1)
        history['train_loss'].append(float(train_loss.data))
        history['validation_loss'].append(float(validation_loss.data))
        
        if early_stopper.early_stop(val_loss_to_monitor):     
            # print(f"Stopping early! {validation_loss}, {epoch}")      
            print(f"Stopping early! {train_loss}, {epoch}")      
            break

    return model,history



In [4]:

# %%
def train_model(hyperparams):

    #Hyperparams
    epochs = int(hyperparams['epochs'])
    patience = int(hyperparams['patience'])
    learning_rate = hyperparams['learning_rate']

    batch_size = int(hyperparams['batch_size'])
    num_dense_layers = int(hyperparams['num_dense_layers'])
    hidden_neurons = int(hyperparams['hidden_neurons'])
    dropout_rate = hyperparams['dropout_rate']
    random_seed = int(hyperparams['random_seed'])
    num_classes = hyperparams['route_info']['num_classes']

    route_id = hyperparams['route_info']['route_id']

    if route_id == 999:
        route_id = ''

    cv_train_loss = []
    cv_val_loss = []

    fold_no = 0
    for fold_no in range(config.CROSS_FOLD_COUNT):
        # LOADING THE DATA 
        data_path = f'{config.path_to_main_folder}/data/graph_data/{config.time_window_ver}_time_window_static_graph/experiment_{config.exp_type}/'
        train_fp = f"{data_path}/{route_id}/train_data_fold_{fold_no}/train_data_fold_{fold_no}_processed_data.parquet"
        val_fp = f"{data_path}/{route_id}/val_data_fold_{fold_no}/val_data_fold_{fold_no}_processed_data.parquet"

        train_data = pd.read_parquet(train_fp)
        val_data = pd.read_parquet(val_fp)
        ohe = joblib.load(f'{data_path}/{route_id}/train_data_fold_{fold_no}/ohe.joblib')

        train_data = train_data[:1000]
        val_data = train_data

        train_data = train_data[REQ_COLS]
        val_data = val_data[REQ_COLS]
                
        data_to_scale = train_data[ohe_columns]
        train_data[ohe.get_feature_names_out()] = ohe.transform(data_to_scale).toarray()

        data_to_scale = val_data[ohe_columns]
        val_data[ohe.get_feature_names_out()] = ohe.transform(data_to_scale).toarray()

        train_data.drop(ohe_columns,axis=1,inplace=True)
        val_data.drop(ohe_columns,axis=1,inplace=True)

        # GETTING THE the class weights used for focal loss
        y_train = np.array(train_data[TARGET])
        class_counts = np.bincount(y_train)
        total_samples = len(y_train)
        class_weights = []
        for count in class_counts:
            weight = 1 / (count / total_samples)
            class_weights.append(weight)

        X_train = train_data[num_columns+cat_columns+list(ohe.get_feature_names_out())] 
        y_train = train_data[TARGET]

        X_val = val_data[num_columns+cat_columns+list(ohe.get_feature_names_out())]
        y_val = val_data[TARGET]

        input_size = X_train.shape[-1]

        train_dataset = TensorDataset(torch.tensor(np.array(X_train)).float() , torch.tensor(np.array(y_train))) # Flatten y for compatibility with DataLoader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        validation_dataset = TensorDataset(torch.tensor(np.array(X_val)).float() , torch.tensor(np.array(y_val))) # Flatten y for compatibility with DataLoader
        validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

        torch.manual_seed(random_seed)
        random.seed(random_seed)
        np.random.seed(random_seed)
        # CREATE THE MODEL
        model = utils.MLPModel(input_size, num_classes, hyperparams)
        model = model.to(device)
        # Setting up the optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        # Setting the class weights to a tensor
        class_weights = torch.FloatTensor(class_weights).to(device)
        # Setting the focal loss according to the class weights 
        focal_loss = torch.hub.load(
            'adeelh/pytorch-multi-class-focal-loss',
            model='FocalLoss',
            alpha=class_weights,
            gamma=2,
            reduction='mean',
            force_reload=False
        ).to(device)

        # TRAIN THE MODEL 
        model, history = train_and_evaluate_model(model, epochs, patience, train_dataset, validation_dataset, focal_loss, optimizer)
        index_val = history['validation_loss'].index(min(history['validation_loss']))
        
        #Storing all the data
        cv_train_loss.append(history['train_loss'][index_val])
        cv_val_loss.append(history['validation_loss'][index_val])

        # Call back to save the best model till now
        # route_id = hyperparams['route_info']['route_id']

        print(f'Training for fold {fold_no} ...')
        fold_no = fold_no+1

    cv_val_loss_mean = mean(cv_val_loss)

    tune.report(
        cv_val_loss_mean=cv_val_loss_mean,
        cv_train_loss=cv_train_loss,
        cv_val_loss=cv_val_loss,
    )

    return model,history


In [5]:

# %%
def train_tune(hyperparams):
    # Start Ray experiment and pass the `hyperparams` to the `train` function
    model,history = train_model(hyperparams)
    print("Saving the model and history")

    def save_model_and_history_callback(model, history, hyperparams):
            epochs = int(hyperparams['epochs'])
            patience = int(hyperparams['patience'])
            learning_rate = hyperparams['learning_rate']
            batch_size = int(hyperparams['batch_size'])
            num_dense_layers = int(hyperparams['num_dense_layers'])
            hidden_neurons = int(hyperparams['hidden_neurons'])
            dropout_rate = hyperparams['dropout_rate']
            random_seed = int(hyperparams['random_seed'])
            num_classes = hyperparams['route_info']['num_classes']
            route_id = hyperparams['route_info']['route_id']    

            hp_filename = f'{epochs}_{patience}_lr_{learning_rate}_bs_{batch_size}_ndl_{num_dense_layers}_hn_{hidden_neurons}_rs_{random_seed}_do_{dropout_rate}'

            folder = f'../models/{hp_filename}'
            pathlib.Path(folder).mkdir(parents=True, exist_ok=True) 

            # Generate a unique file name for each trial
            file_name = f"{folder}/model_{hp_filename}.pt"    
            # Save the model to disk
            torch.save({
                'epoch': epochs,
                'model_state_dict': model.state_dict(),
            }, file_name)
            
            # Save the history to disk
            history_file_name = f"{folder}/history_{hp_filename}.json"
            with open(history_file_name, 'w') as f:
                json.dump(history, f)
                
    save_model_and_history_callback(model,history,hyperparams)

In [6]:

# %%
# Function for ray hyperparameter tuning 
def ray_hyper_parameter_tune(route_info):

    # TODO: FIX BELOW 
    hyperparams = {
        'epochs':   tune.quniform(100, 600, 1),
        'patience': tune.quniform(50,150,1),
        'learning_rate' : tune.uniform(0.0001, 0.1),
        'batch_size' : tune.quniform(64,128,1),
        'num_dense_layers': tune.quniform(1, 5, 1),
        'hidden_neurons': tune.quniform(1,256,1),
        'dropout_rate' : tune.uniform(0,0.5),
        'random_seed': tune.quniform(0, 300, 1),
        'route_info': route_info
    }

    dir_fp = f"ray_tune/hyperparams_output/STL_{config.version}/"
    local_dir = f'../{dir_fp}'
    pathlib.Path(local_dir).mkdir(parents=True, exist_ok=True) 

    ray.init()

    path_to_local_dir = f'{config.path_to_main_folder}/{dir_fp}'

    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0})
    algo = ConcurrencyLimiter(algo, max_concurrent=3)
   
    analysis = tune.run(
            #  
            # If you want to use other parameters, you can add them below like tune.with_parameters(train_tune,train_dataset=train_dataset,validation_dataset=validation_dataset) . Function: def train_tune(hyperparams,train_dataset,validation_dataset)
            tune.with_parameters(train_tune),
            config=hyperparams,
            resources_per_trial={"cpu": 4},
            metric="cv_val_loss_mean",
            mode="min",
            search_alg=algo,
            num_samples=config.RAY_NUM_SAMPLES,
            # max_concurrent_trials=5,
            local_dir= f'{path_to_local_dir}',
            name=f"experiment_STL_{config.version}_{route_info['route_id']}",
            max_failures=1,
            raise_on_failed_trial=False
        )

    # Stop Ray
    ray.shutdown()

    temp = analysis.dataframe()   
    hyperparams_path = f'{local_dir}'
    pathlib.Path(hyperparams_path).mkdir(parents=True, exist_ok=True) 
    temp.to_csv(f"{hyperparams_path}/RAY_RESULTS_STL_{config.version}_ROUTE_{route_info['route_id']}.csv")


In [7]:
# route_info = {} 

# route_info['route_id'] = 999
# route_info['num_classes'] = 5

# hyperparams = {
#         'epochs':   10,
#         'patience': 3,
#         'learning_rate' : 0.01,
#         'GNN_hidden_channels': 36,
#         'gru_layers': 2,
#         'chebyshev_filter': 3,
#         'linear_layer_count': 3,
#         'linear_hidden_channels': 20,
#         'dropout_rate' : 0.1,
#         'route_info': route_info
#     }
# model,history = train_model(hyperparams)


In [8]:
route_info = {} 

route_info['route_id'] = 55
route_info['num_classes'] = 4


ray_hyper_parameter_tune(route_info)

ray.shutdown()

2024-02-28 19:26:41,981	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2024-02-28 19:26:42,694	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/Users/samirgupta/Desktop/WORK/--_work/Research/After_12th_December_2023/05_GNN_occu_prediction/WeGo_Occu_pred/NEW_FOLDER/CODE/venv/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2024-02-28 19:26:42,704	WARNING bayesopt_search.py:421 -- BayesOpt search does not support quantization. Dropped quantization.
2024-02-28 19:26:42,704	WARNING bayesopt_search.py:431 -- BayesOpt does not support specific sampling methods. The Uniform sampler 

2024-02-28 19:26:44,230	WARNING worker.py:2006 -- Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 1418, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1498, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "/Users/samirgupta/Desktop/WORK/--_work/Research/After_12th_December_2023/05_GNN_occu_prediction/WeGo_Occu_pred/NEW_FOLDER/CODE/venv/lib/python3.10/site-packages/ray/_private/function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/Users/samirgupta/Desktop/WORK/--_work/Research/After_12th_December_2023/05_GNN_occu_prediction/WeGo_Occu_pred/NEW_FOLDER/CODE/venv/lib/python3.10/site-packages/ray/_private/function_manager.py", line 638, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to i

Trial name
train_tune_2b9e5c13


2024-02-28 19:26:46,869	WARNING worker.py:2006 -- Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 1418, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1498, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "/Users/samirgupta/Desktop/WORK/--_work/Research/After_12th_December_2023/05_GNN_occu_prediction/WeGo_Occu_pred/NEW_FOLDER/CODE/venv/lib/python3.10/site-packages/ray/_private/function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/Users/samirgupta/Desktop/WORK/--_work/Research/After_12th_December_2023/05_GNN_occu_prediction/WeGo_Occu_pred/NEW_FOLDER/CODE/venv/lib/python3.10/site-packages/ray/_private/function_manager.py", line 638, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to i